In [1]:
import torch
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict
import json
import os
from typing import Dict, List, Tuple, Optional
# from tqdm.auto import tqdm
import datetime
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer
import traceback

In [2]:
def get_device():
    """Get the optimal available device"""
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        # Enable TF32 for better performance on Ampere GPUs (A100, A6000, etc)
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        # Set memory allocation settings
        torch.cuda.empty_cache()
        # Enable CUDNN benchmarking for better performance
        torch.backends.cudnn.benchmark = True
    else:
        device = torch.device("cpu")
    return device

In [3]:
def load_model():
    device = get_device()
    model = AutoModelForCausalLM.from_pretrained('deepseek-ai/deepseek-moe-16b-base',
                                                trust_remote_code=True,
                                                torch_dtype=torch.float16)
    model = model.to(device)
    tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/deepseek-moe-16b-base',
                                                trust_remote_code=True)
    return model, tokenizer

model, tokenizer = load_model()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
def prepare_text_chunks(file_path: str, chunk_size: int = 2048, tokenizer = None) -> List[str]:
    """
    Split input text file into chunks of specified token length with memory efficiency
    """
    chunks = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            if tokenizer:
                # Process file in chunks to avoid loading entire file
                chunk_text = ""
                buffer = []
                
                for line in f:
                    buffer.append(line)
                    # Join buffer and check token length
                    test_text = "".join(buffer)
                    tokens = tokenizer.encode(test_text)
                    
                    # If we exceed max length, process the buffer
                    if len(tokens) >= chunk_size:
                        # Take only chunk_size tokens
                        chunk_tokens = tokens[:chunk_size]
                        chunks.append(tokenizer.decode(chunk_tokens))
                        
                        # Keep remainder for next chunk
                        remainder_tokens = tokens[chunk_size:]
                        buffer = [tokenizer.decode(remainder_tokens)]
                
                # Add remaining text if any
                if buffer:
                    final_text = "".join(buffer)
                    final_tokens = tokenizer.encode(final_text)
                    if len(final_tokens) > chunk_size:
                        # Split remaining text into chunks of chunk_size
                        for i in range(0, len(final_tokens), chunk_size):
                            chunk_tokens = final_tokens[i:i + chunk_size]
                            chunks.append(tokenizer.decode(chunk_tokens))
                    else:
                        chunks.append(final_text)
            else:
                words = []
                for line in f:
                    words.extend(line.split())
                    while len(words) >= chunk_size:
                        chunk = ' '.join(words[:chunk_size])
                        chunks.append(chunk)
                        words = words[chunk_size:]
                if words:
                    chunks.append(' '.join(words))
                    
        print(f"Created {len(chunks)} chunks from the input file")
        return chunks
        
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

In [5]:
@torch.no_grad()
def get_expert_distribution(model, input_text: str, tokenizer, device: torch.device, 
                          num_shared_experts: int = 0, num_topk: int = 8) -> Dict[int, List[Tuple[str, int, float]]]:
    """
    Get expert routing probabilities with proper handling of shared experts and top-k tracking
    """
    try:
        model = model.to(device)
        
        inputs = tokenizer(input_text, 
                         return_tensors="pt",
                         padding=True,
                         truncation=True,
                         max_length=2048)
        
        inputs = {k: v.to(device) for k, v in inputs.items()} 
        
        outputs = model(**inputs, 
                       output_attentions=False, 
                       output_hidden_states=True, 
                       return_dict=True)
        
        hidden_states = outputs.hidden_states
        layer_distributions = {}

        # Process each MoE layer
        for layer_idx, layer in enumerate(model.model.layers[1:], 1):
            moe_layer = layer.mlp
            layer_hidden = hidden_states[layer_idx]

            # Initialize distributions
            expert_distributions = []
            
            # Convert token IDs to text
            tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            # Get router logits for dynamic experts
            router_logits = torch.matmul(
                layer_hidden.float(),
                moe_layer.gate.weight.t().float()
            )
            
            # Get top-k routed experts
            router_probs = torch.softmax(router_logits, dim=-1)
            top_probs, top_indices = torch.topk(router_probs, k=num_topk, dim=-1)

            # Process results for each token
            for i in range(len(tokens)):
                if tokens[i] == tokenizer.pad_token:
                    continue
                    
                # Skip special tokens like begin/end of sentence
                if tokens[i].startswith('<') and tokens[i].endswith('>'):
                    continue
                    
                clean_token = tokens[i].replace('Ġ', '')
                
                # Add shared experts for each token (they process all tokens)
                for shared_idx in range(num_shared_experts):
                    expert_distributions.append(
                        (clean_token, shared_idx, 1.0)  # Shared experts always have weight 1.0
                    )
                
                # Add routed experts
                for j in range(num_topk):
                    expert_distributions.append(
                        (clean_token, 
                         int(top_indices[0,i,j].item()) + num_shared_experts,  # Offset expert indices by num shared experts
                         float(top_probs[0,i,j].item()))
                    )
            
            layer_distributions[layer_idx] = expert_distributions

        return layer_distributions

    except Exception as e:
        print(f"Error in get_expert_distribution: {e}")
        return {}

In [6]:
def save_expert_stats(distributions: Dict[int, List[Tuple[str, int, float]]], 
                     domain: str,
                     output_dir: str = 'expert_stats'):
    """Save expert routing statistics with error handling"""
    try:
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f'{domain}_expert_stats.json')
        
        stats = {}
        for layer_idx, layer_data in distributions.items():
            expert_counts = defaultdict(int)
            total = len(layer_data)
            
            if total == 0:
                continue
                
            for _, expert_idx, _ in layer_data:
                expert_counts[expert_idx] += 1
                
            expert_probs = {
                str(expert): count/total 
                for expert, count in expert_counts.items()
            }
            stats[str(layer_idx)] = expert_probs
        
        with open(output_path, 'w') as f:
            json.dump(stats, f, indent=2)
            
    except Exception as e:
        print(f"Error saving expert stats: {e}")

In [7]:
def analyze_token_distributions(expert_distributions):
    """
    Convert expert distributions to token counts per expert per layer
    
    Args:
        expert_distributions: Dictionary mapping layer_id to list of (token, expert_id, probability) tuples
        
    Returns:
        Dictionary mapping layer_id to dict of expert_id: token_count
    """
    token_counts = {}
    
    # Process each layer
    for layer_id, distributions in expert_distributions.items():
        expert_counts = defaultdict(int)
        
        # Count tokens per expert in this layer, including shared experts
        for token, expert_id, prob in distributions:
            expert_counts[expert_id] += 1
            
        token_counts[layer_id] = dict(expert_counts)
        
    return token_counts

In [34]:
def plot_layer_distribution(token_counts, layer_id: int, domain: str, dark_mode: bool = True, tokenizer=tokenizer):
    """
    Create a bar graph showing token distribution across experts for a specific layer
    """
    try:
        # Get expert counts for specified layer
        layer_counts = token_counts.get(layer_id, {})
        if not layer_counts:
            raise ValueError(f"No data found for layer {layer_id}")
            
        # Create lists for x and y values
        expert_ids = sorted(layer_counts.keys())
        token_counts_list = [layer_counts[expert_id] for expert_id in expert_ids]
        
        # Create labels for experts
        expert_labels = [f"{expert_id}" for expert_id in expert_ids]
        
        # Calculate totals
        total_routing_decisions = sum(token_counts_list)
        
        file_path = f"dataset/{domain}.txt"
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        tokens = tokenizer(text)
        actual_tokens = len(tokens['input_ids'])

        # Create hover text
        hover_text = [f"Expert {expert_id}<br>Tokens: {count}<br>Percentage: {(count/total_routing_decisions*100):.1f}%" 
                     for expert_id, count in zip(expert_ids, token_counts_list)]

        # Create bar graph
        fig = go.Figure(data=[
            go.Bar(
                x=expert_labels,
                y=token_counts_list,
                hovertext=hover_text,
                hoverinfo='text',
                marker_color='#636EFA'
            )
        ])
        
        # Update layout
        template = 'plotly_dark' if dark_mode else 'plotly_white'
        fig.update_layout(
            title=f'Token Distribution Across Experts in Layer {layer_id} ({domain})',
            xaxis_title='Expert',
            yaxis_title='Number of Routing Decisions',
            width=1200,
            height=600,
            template=template,
            showlegend=False,
            xaxis={'tickangle': -45},
            yaxis=dict(range=[0, total_routing_decisions])  # Set max on y-axis to total routing decisions
        )
        
        # Add annotations for both total routing decisions and actual tokens
        fig.add_annotation(
            text=f'Total Routing Decisions: {total_routing_decisions:,}<br>Actual Tokens: {actual_tokens:,}',
            xref='paper', yref='paper',
            x=1, y=1,
            xanchor='right', yanchor='top',
            showarrow=False,
            font=dict(size=12)
        )
        
        return fig
        
    except Exception as e:
        print(f"Error creating bar plot: {e}")
        return None

In [9]:
def plot_token_heatmap(token_counts, domain, n_experts=64, n_layers=27, num_shared_experts=1):
    """
    Create heatmap of token counts per expert per layer, excluding shared experts
    
    Args:
        token_counts: Dictionary mapping layer_id to dict of expert_id: token_count
        n_experts: Total number of experts
        n_layers: Total number of layers
        domain: domain name for heatmap title
        num_shared_experts: Number of shared experts to exclude from visualization
    """
    # Initialize count matrix
    count_matrix = np.zeros((n_layers, n_experts-num_shared_experts))  # Reduced size to exclude shared experts
    
    # Fill matrix with token counts, skipping shared experts
    for layer_id, expert_counts in token_counts.items():
        for expert_id, count in expert_counts.items():
            if expert_id >= num_shared_experts and expert_id < n_experts:  # Skip shared experts
                count_matrix[layer_id-1, expert_id-num_shared_experts] = count  # Adjust index
    
    # Create heatmap with adjusted expert indices
    fig = go.Figure(data=go.Heatmap(
        z=count_matrix,
        x=[f'{i+num_shared_experts}' for i in range(n_experts-num_shared_experts)],  # Start after shared experts
        y=[f'{i}' for i in range(1, n_layers+1)],
        colorscale='Viridis',
        hoverongaps=False,
        hovertemplate='Layer %{y}<br>Expert %{x}<br>Tokens: %{z:d}<extra></extra>'
    ))
    
    fig.update_layout(
        title=f'token distribution heatmap for {domain}',
        xaxis_title='Expert Index',
        yaxis_title='Layer',
        width=1200,
        height=800,
        template='plotly_dark'
    )
    
    return fig, count_matrix

In [10]:
def analyze_dataset(
    plot_type: str = 'heatmap',
    file_path: str = None,
    model = None,
    tokenizer = None,
    domain: str = 'test1',
    chunk_size: int = 2048,
    num_shared_experts: int = 2,  
    num_topk: int = 6,
    layer_id: int = 1,
    device: torch.device = None,
    force_recompute: bool = False  # New parameter to force recomputation
    ):
    """
    Main analysis function with caching of computed results
    """
    try:
        # Check for existing results
        npy_path = f'plots-deepseek/{domain}_count_matrix.npy'
        if os.path.exists(npy_path) and not force_recompute:
            print(f"Loading existing results from {npy_path}")
            count_matrix = np.load(npy_path)
            
            # Reconstruct token_counts from count_matrix
            token_counts = {}
            for layer in range(count_matrix.shape[0]):
                expert_counts = {}
                for expert in range(count_matrix.shape[1]):
                    if count_matrix[layer, expert] > 0:
                        expert_counts[expert + 1] = int(count_matrix[layer, expert])  # Add 1 to expert index
                token_counts[layer + 1] = expert_counts
            
            # Create visualization based on plot_type
            print("Generating visualization from cached data...")
            if plot_type == 'heatmap':
                fig, _ = plot_token_heatmap(
                    token_counts,
                    n_experts=64,
                    n_layers=27,
                    domain=domain
                )
            else:  # plot_type == 'bar'
                fig = plot_layer_distribution(
                    token_counts=token_counts,
                    layer_id=layer_id,
                    domain=domain
                )
            
            return fig, count_matrix, token_counts
            
        # If no cached results or force_recompute is True, proceed with full computation
        print("No cached results found or force_recompute=True. Computing from scratch...")
        
        # Rest of the original function remains the same
        if device is None:
            device = get_device()
        print(f"Using device: {device}")
        
        model = model.to(device)
        
        if plot_type not in ['heatmap', 'bar']:
            raise ValueError("plot_type must be either 'heatmap' or 'bar'")
        
        # Process in smaller chunks to avoid OOM
        max_chunk_tokens = 1024
        chunks = prepare_text_chunks(file_path, max_chunk_tokens, tokenizer)
        if not chunks:
            raise ValueError("No text chunks generated")
            
        # Process chunks
        all_distributions = defaultdict(list)
        total_chunks = len(chunks)
        
        for i, chunk in enumerate(chunks):
            torch.cuda.empty_cache()
            print(f"Processing chunk {i+1}/{total_chunks}")
            
            chunk_dist = get_expert_distribution(
                model, 
                chunk,
                tokenizer,
                device,
                num_shared_experts=num_shared_experts,
                num_topk=num_topk
            )
            
            for layer, layer_data in chunk_dist.items():
                all_distributions[layer].extend(
                    [(t, e, float(p)) for t, e, p in layer_data]
                )
                
        token_counts = analyze_token_distributions(all_distributions)
        
        print("Generating visualization...")
        if plot_type == 'heatmap':
            fig, count_matrix = plot_token_heatmap(
                token_counts,
                n_experts=64,
                n_layers=27,
                domain=domain
            )
        else:
            fig = plot_layer_distribution(
                token_counts=token_counts,
                layer_id=layer_id,
                domain=domain
            )
            count_matrix = None
        
        # Save results
        os.makedirs('plots-deepseek', exist_ok=True)
        if plot_type == 'heatmap':
            np.save(npy_path, count_matrix)
            fig.write_html(f'plots-deepseek/{domain}_heatmap.html')
            fig.write_image(f'plots-deepseek/{domain}_heatmap.png')
        else:
            fig.write_html(f'plots-deepseek/{domain}_layer{layer_id}_bargraph.html')
            fig.write_image(f'plots-deepseek/{domain}_layer{layer_id}_bargraph.png')
        
        print("Analysis complete!")
        return fig, count_matrix, token_counts
        
    except Exception as e:
        print(f"Error in analyze_dataset: {e}")
        traceback.print_exc()
        return None, None, None

### plotting

In [25]:
# Analyze with proper bounds checking
fig, count_matrix, token_counts = analyze_dataset(
    plot_type='heatmap', # or 'bar'
    file_path='dataset/humaneval.txt',
    model= model,
    tokenizer= tokenizer,
    domain='humaneval',
    chunk_size=1024,
    num_shared_experts=1, 
    num_topk=7,
    force_recompute=False
    # layer_id=1 # only for bar plot
)

fig.show()

Loading existing results from plots-deepseek/humaneval_count_matrix.npy
Generating visualization from cached data...


In [24]:
fig, count_matrix, token_counts = analyze_dataset(
    plot_type='heatmap', # or 'bar'
    file_path='dataset/gsm8k.txt',
    model= model,
    tokenizer= tokenizer,
    domain='gsm8k',
    chunk_size=1024,
    num_shared_experts=1, 
    num_topk=7,
    force_recompute=False
    # layer_id=1 # only for bar plot
)

fig.show()

Loading existing results from plots-deepseek/gsm8k_count_matrix.npy
Generating visualization from cached data...


In [23]:
fig, count_matrix, token_counts = analyze_dataset(
    plot_type='heatmap', # or 'bar'
    file_path='dataset/piqa.txt',
    model= model,
    tokenizer= tokenizer,
    domain='piqa',
    chunk_size=1024,
    num_shared_experts=1, 
    num_topk=7,
    force_recompute=False
    # layer_id=1 # only for bar plot
)

fig.show()

Loading existing results from plots-deepseek/piqa_count_matrix.npy
Generating visualization from cached data...


In [22]:
fig, count_matrix, token_counts = analyze_dataset(
    plot_type='heatmap', # or 'bar'
    file_path='dataset/arc_easy.txt',
    model= model,
    tokenizer= tokenizer,
    domain='arc_easy',
    chunk_size=1024,
    num_shared_experts=1, 
    num_topk=7,
    force_recompute=False
    # layer_id=1 # only for bar plot
)

fig.show()

Loading existing results from plots-deepseek/arc_easy_count_matrix.npy
Generating visualization from cached data...


In [40]:
fig, count_matrix, token_counts = analyze_dataset(
    plot_type='bar',
    file_path='dataset/arc_easy.txt',
    model= model,
    tokenizer= tokenizer,
    domain='arc_easy',
    chunk_size=1024,
    num_shared_experts=1, 
    num_topk=7,
    force_recompute=False,
    layer_id=1 # only for bar plot
)

fig.show()

Loading existing results from plots-deepseek/arc_easy_count_matrix.npy
Generating visualization from cached data...
